In [19]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
from astropy.coordinates import EarthLocation
import astropy.units as u
# from icecream import ic, install

from astropaul.database import database_connection
import astropaul.targetlistcreator as tlc
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

# install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
html_dir = "../../../../Observing Files/Fan Mountain 2025"
html.clear_directory(html_dir)
session = tlc.ObservingSession(
    ap.Observer(EarthLocation(lat="37:52:41.35", lon="-78:41:34.92", height=566 * u.m), timezone="utc", name="Fan Mountain")
)
session.add_day_range("2025-08-29", "2025-12-31")

phase_event_defs = [
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

fan_targets = [
    # target of interest
    "TIC 344541836",
    # wide, bright QEBs 
    "TIC 278465736",
    "TIC 470710327",
    "TIC 367448265",
    "TIC 317863971",
    "TIC 252980672",
    "TIC 159089190",
    "TIC 70664819",
    "TIC 286470992",
    "TIC 307119043",
    "TIC 200094011",
]

# fan_targets = [
#     # wide, bright QEBs
#     "TIC 278465736",
#     "TIC 470710327",
#     "TIC 367448265",
#     "TIC 376606423",
#     "TIC 317863971",
#     "TIC 159089190",
#     "TIC 70664819",
#     "TIC 286470992",
#     "TIC 78568780",
#     "TIC 200094011",
# ]

min_altitude = 35 * u.deg

with database_connection() as conn:
    name = "Fan Mountain SIDE Summer 2025"
    creator = tlc.TargetListCreator(name=name, connection=conn, phase_event_defs=phase_event_defs)
    creator.steps = [
        tlc.add_targets,
        partial(tlc.filter_targets, criteria=lambda df: (df["Target Name"].isin(fan_targets))),
        tlc.ancillary_data_from_tess,
        partial(
            tlc.add_observability,
            observing_session=session,
            calc_moon_distance=True,
            observability_threshold=(min_altitude, 80 * u.deg),
        ),
        partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
        tlc.add_tess_catalog_associations,
        tlc.add_lists,
        partial(tlc.add_database_table, table_name="ephemerides"),
        partial(tlc.add_database_table, table_name="dssi_observations"),
        partial(tlc.add_database_table, table_name="speckle_detections"),
        partial(
            tlc.add_system_configuration,
            table_name="DSSI Observations",
            time_column="Mid JD",
            eclipse_table="SIDE Observations",
        ),
        partial(tlc.add_database_table, table_name="pepsi_observations"),
        tlc.add_pepsi_evaluations,
        partial(tlc.add_system_configuration, table_name="PEPSI Observations", time_column="Mid JD"),
        partial(tlc.add_phase_events, observing_session=session, event_types=["Eclipse", "Not in Eclipse"]),
    ]
    tl = creator.calculate(verbose=False)

# html.render_observing_pages(tl, None, {}, html_dir)
print(tl.summarize())

C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:343: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  overall_every_night &= segment_observability
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:346: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  overall_min_hours_observable = [
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:357: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

Name: Fan Mountain SIDE Summer 2025
Criteria
    lambda df: (df["Target Name"].isin(['TIC 344541836', 'TIC 278465736', 'TIC 470710327', 'TIC 367448265', 'TIC 317863971', 'TIC 252980672', 'TIC 159089190', 'TIC 70664819', 'TIC 286470992', 'TIC 307119043', 'TIC 200094011']))
    AltitudeConstraint: {'min': 35.0, 'max': 80.0, 'boolean_constraint': True}
    lambda df: (df["Observable Any Night"])
11 targets:
      11 QuadEB
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 123)
    Observable: (5, 500)
    List: (0, 14)
    Count: (5, 0)
Associated tables:
     125 rows,  2 columns: Lunar Phases
      54 rows,  3 columns: Catalog Membership
      55 rows,  2 columns: List Memberships
      34 rows,  7 columns: Ephemerides
      49 rows, 12 columns: DSSI Observations
      82 rows, 11 columns: Speckle Detections
      23 rows,  6 columns: SIDE Observations
      46 rows, 18 columns: PEPSI Observations
    7249 rows, 10 columns: Phase Events



In [62]:
altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, True)
# pr.prioritize_side_observation(pl, side_state="Eclipse")
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.0)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, html_dir)
# pl.categorical_priorities[0]